# Bokeh - interactive visualization

## 1. Introduction

Bokeh is a Python library for interactive visualization on web browsers. It provides interactive plots, dashboards, and data applications for users that helps people easily know figures or diagram. This spotlight will tell you how to use bokeh. In addition, there are some example that can explain interactive data visualization in Python with Bokeh.<br>
I would like to implement PageRank algorithm and visualize directed graph via Bokeh. User can move mouse on each node to know more information. Also, there are some features that can be used to report information, to change plot parameters such as zoom level or range extents, or to add, edit, or delete glyphs.

## 2. Installation

1. pip install bokeh  
2. pip install -i https://pypi.tuna.tsinghua.edu.cn/simple bokeh

## 3. From Data to Visualization

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import bokeh.io as bkio
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.palettes as bkpal
import pygraphviz
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool, MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,ColumnDataSource, LabelSet)
from bokeh.transform import transform    
from bokeh.models import CustomJSTransform, LabelSet
import math
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.palettes import Category20c,Spectral4
from bokeh.io import output_notebook

In [2]:
G = nx.DiGraph()      
                      #0.             1.                2.         3.           4.       
for name, labels in [("a", {1, 2}), ("b", {2, 3, 4}), ("c", {4}),("d", {-1}), ("e", {3})]:
    G.add_node(name, labels=labels)
for src, tgt in ["ab", "ac", "bc", "bd", "be", "ce", "ed"]:
    G.add_edge(src, tgt, labels=G.node[src]["labels"] | G.node[tgt]["labels"])

nodes=pd.DataFrame.from_dict(nx.drawing.nx_pydot.pydot_layout(G,prog='dot'),orient="index", columns=["x", "y"])

### Use PageRank in Networkx

In [3]:
pr = nx.pagerank(G, alpha=0.9)
PR = pd.DataFrame(list(pr.items()))
PR.columns = ['Nodes','Probs']
print(PR)

  Nodes     Probs
0     a  0.085325
1     b  0.123722
2     c  0.160839
3     d  0.362918
4     e  0.267196


In [4]:
# 1.2. normalize layout in [0;1]
for coord in ["x", "y"] :
    nodes[coord] -= nodes[coord].min()
    nodes[coord] /= nodes[coord].max()

In [5]:
# 1.3. concert labels to str
nodes["labels"] = [", ".join(sorted(str(l) for l in G.node[node]["labels"])) for node in nodes.index]
# 1.4. create edges dataframe
idx = nodes.to_dict(orient="index")
edges = pd.DataFrame(((src, idx[src]["x"], idx[src]["y"],
                        tgt, idx[tgt]["x"], idx[tgt]["y"],
                        ", ".join(sorted(str(l) for l in label)))
                        for src, tgt, label in G.edges(data="labels")), 
                        columns=["start", "x_start", "y_start","end", "x_end", "y_end","labels"])

In [6]:
nodes_size = 0.02
arrow_size = 0.01
# 1.5. compute edges slopes
edges["slope"] = np.arctan2(edges["y_end"]-edges["y_start"], edges["x_end"]-edges["x_start"])
# 1.6. compute edges start/end points wrt to nodes size
edges["x_start"] += np.cos(edges["slope"]) * nodes_size
edges["y_start"] += np.sin(edges["slope"]) * nodes_size
edges["x_end"] -= np.cos(edges["slope"]) * nodes_size
edges["y_end"] -= np.sin(edges["slope"]) * nodes_size
edges["size"] = np.full(len(edges), arrow_size)

### draw graph with bokeh

In [8]:
# 2.1. get palette
pal = getattr(bkpal, "Spectral4", bkpal.Spectral4)
# 2.2. create plot
plot = bkp.figure(title="Graph", x_range=(-nodes_size, 1 + nodes_size), 
                  y_range=(-nodes_size, 1 + nodes_size), x_axis_location=None, 
                  y_axis_location=None, outline_line_dash='dotted')

In [9]:
# 2.3. create graph renderer
graph = bkm.GraphRenderer()
# 2.4. create and add nodes CDS
nodes_dict = {k: nodes[k] for k in nodes.columns}
nodes_dict["index"] = nodes.index
nodes_data = bkm.ColumnDataSource(data=nodes_dict)
graph.node_renderer.data_source.data = nodes_data.data

In [10]:
# 2.5. create and add edges CDS
edges_dict = {k: edges[k] for k in edges.columns}
edges_dict["xs"] = [(edges["x_start"][e], edges["x_end"][e]) for e in edges.index]
edges_dict["ys"] = [(edges["y_start"][e], edges["y_end"][e]) for e in edges.index]
edges_data = bkm.ColumnDataSource(data=edges_dict)
graph.edge_renderer.data_source.data = edges_data.data

In [11]:
# 2.6. create layout as expected by GraphRenderer
layout = {n : (nodes["x"][n], nodes["y"][n]) for n in nodes.index}
graph.layout_provider = bkm.StaticLayoutProvider(graph_layout=layout)
# 2.7. add arrows
arrows = bkm.Arrow(end=bkm.NormalHead(size=arrow_size*1000), x_start="x_start", y_start="y_start", 
                   x_end="x_end", y_end="y_end", source=edges_data)
plot.add_layout(arrows)

graph.node_renderer.glyph = bkm.Circle(radius=nodes_size, fill_color=pal[0], fill_alpha=.5)
graph.node_renderer.selection_glyph = bkm.Circle(radius=nodes_size, fill_color=pal[2], fill_alpha=.5)
graph.node_renderer.hover_glyph = bkm.Circle(radius=nodes_size, fill_color=pal[1], fill_alpha=.5)


graph.edge_renderer.glyph = bkm.MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph.edge_renderer.selection_glyph = bkm.MultiLine(line_color=pal[2], line_width=5)
graph.edge_renderer.hover_glyph = bkm.MultiLine(line_color=pal[1], line_width=5)
# 2.9. set policies and finish it
graph.selection_policy = bkm.NodesAndLinkedEdges()
graph.inspection_policy = bkm.NodesAndLinkedEdges()
plot.renderers.append(graph)
plot.add_tools(bkm.TapTool(), bkm.LassoSelectTool(), bkm.BoxSelectTool(),
               bkm.HoverTool(tooltips=[("node", "@index"),("labels", "@labels")]))
#     bkio.output_file(“interactive_graphs.html”)
output_notebook()
# add the labels to the node renderer data source
source = graph.node_renderer.data_source

source.data['names'] = [str(x) for x in source.data['index']]
code = """
    var result = new Float64Array(xs.length)
    for (var i = 0; i < xs.length; i++) {
        result[i] = provider.graph_layout[xs[i]][%s]
    }
    return result
"""
xcoord = CustomJSTransform(v_func=code % "0", args=dict(provider=graph.layout_provider))
ycoord = CustomJSTransform(v_func=code % "1", args=dict(provider=graph.layout_provider))

# Use the transforms to supply coords to a LabelSet 
labels = LabelSet(x=transform('index', xcoord),
                  y=transform('index', ycoord),
                  text='names', text_font_size="16px",
                  x_offset=15, y_offset=5,
                  source=source, render_mode='canvas')

plot.add_layout(labels)

show(plot)

Loading BokehJS ...

### Bar Graph for PageRank probabilities

In [12]:
xdata = PR['Nodes']
ydata = PR['Probs']
p = figure(x_range=xdata, plot_height=350, title="Bar graph @ bokeh", 
           x_axis_label='Nodes', y_axis_label='Probability')
p.vbar(x=xdata, top=ydata, width=0.9)
show(p)

In [13]:
# def bokeh_plot3(nodes,res):
xdata = PR['Nodes']
ydata = PR['Probs']
x=dict(zip(xdata, ydata))
data = pd.Series(x).reset_index(name='value').rename(columns={'index':'node'})

data['angle'] = data['value'] / data['value'].sum() * 2 * math.pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="Pie Chart",tooltips="@node: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='node', source=data)
show(p)